In [1]:
import tensorflow as tf
import gc
import os
import numpy as np
import tensorflow_datasets as tfds

In [2]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))

All devices:  [LogicalDevice(name='/device:TPU:0', device_type='TPU'), LogicalDevice(name='/device:TPU:1', device_type='TPU'), LogicalDevice(name='/device:TPU:2', device_type='TPU'), LogicalDevice(name='/device:TPU:3', device_type='TPU'), LogicalDevice(name='/device:TPU:4', device_type='TPU'), LogicalDevice(name='/device:TPU:5', device_type='TPU'), LogicalDevice(name='/device:TPU:6', device_type='TPU'), LogicalDevice(name='/device:TPU:7', device_type='TPU')]


In [3]:
strategy = tf.distribute.TPUStrategy(resolver)

In [4]:
!pip install hickle
import hickle as hkl
import os
import matplotlib.pyplot as plt
from keras import layers,Input,Sequential
from keras.layers import Dense,Flatten,Reshape,Conv2DTranspose,Conv2D,MaxPooling2D,Reshape,Resizing,Dropout,UpSampling2D
from tensorflow.keras import datasets, layers, models, losses, Model
from random import randint
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential,Model
import matplotlib.pyplot as plt
from google.colab  import drive
drive.mount("/gdrive")
%cd /gdrive

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 1.5 MB/s eta 0:00:00
Mounted at /gdrive
/gdrive


In [6]:
with strategy.scope():
    load_locally = tf.saved_model.LoadOptions(experimental_io_device='/job:localhost')
    reconstructed_front =   tf.keras.models.load_model('/gdrive/My Drive/Model_run_v1/Frontal', options=load_locally)
    reconstructed_left=     tf.keras.models.load_model('/gdrive/My Drive/Model_run_v1/Left', options=load_locally)
    reconstructed_right =   tf.keras.models.load_model('/gdrive/My Drive/Model_run_v1/Right', options=load_locally)

In [7]:
load_hkl = hkl.load("/gdrive/My Drive/Yedek/Autoencoder_Datasets_filtered_v1/Static/Frontal.hkl")
X_train_frontal=load_hkl['xtrain'] / 255.
X_test_frontal=load_hkl['xtest'] / 255.
y_train_frontal=load_hkl['ytrain']
y_test_frontal=load_hkl['ytest']
del load_hkl
gc.collect()

431

In [8]:
load_hkl = hkl.load("/gdrive/My Drive/Yedek/Autoencoder_Datasets_filtered_v1/Static/Left.hkl")
X_train_left=load_hkl['xtrain'] / 255.
X_test_left=load_hkl['xtest'] / 255.

del load_hkl
gc.collect()


0

In [9]:

load_hkl = hkl.load("/gdrive/My Drive/Yedek/Autoencoder_Datasets_filtered_v1/Static/Right.hkl")
X_train_right=load_hkl['xtrain'] / 255.
X_test_right=load_hkl['xtest'] / 255.

del load_hkl
gc.collect()

0

In [10]:
xm_train_frontal=reconstructed_front.encoder(X_train_frontal)
xm_test_frontal=reconstructed_front.encoder(X_test_frontal)
del X_train_frontal
del X_test_frontal
gc.collect()

0

In [11]:
xm_train_left=reconstructed_left.encoder(X_train_left)
xm_test_left=reconstructed_left.encoder(X_test_left)
del X_train_left
del X_test_left
gc.collect()

0

In [12]:
xm_train_right=reconstructed_right.encoder(X_train_right)
xm_test_right=reconstructed_right.encoder(X_test_right)
del X_train_right
del X_test_right
gc.collect()

0

# **Fusion of front, left,right,left45, and right45 vectors**

In [20]:
class CNN_Views(Model):
    def __init__(self):
        super(CNN_Views, self).__init__()
        self.front = tf.keras.Sequential([
            layers.Input(shape=xm_train_frontal.shape[1:]),
            layers.Conv2D(16, 3, strides=1, padding='same', activation='relu'),
            layers.MaxPooling2D(2, strides=2),
            layers.Conv2D(8, 3, strides=1, padding='same', activation='relu'),
            layers.MaxPooling2D(2, strides=2),
            layers.Conv2D(4, 3, strides=1, padding='same', activation='relu'),
            layers.Flatten(),
            layers.Dense(20,activation='relu'),
            layers.Dropout(0.1),
            layers.Dense(10,activation='relu')])

        self.left = tf.keras.Sequential([
            layers.Input(shape=xm_train_left.shape[1:]),
            layers.Conv2D(8, 3, strides=1, padding='same', activation='relu'),
            layers.MaxPooling2D(2, strides=2),
            layers.Conv2D(4, 3, strides=1, padding='same', activation='relu'),
            layers.MaxPooling2D(2, strides=2),
            layers.Conv2D(2, 3, strides=1, padding='same', activation='relu'),
            layers.Flatten(),
            layers.Dense(10,activation='relu'),
            layers.Dropout(0.2),
            layers.Dense(5,activation='relu')])

        self.right = tf.keras.Sequential([
            layers.Input(shape=xm_train_right.shape[1:]),
            layers.Conv2D(8, 3, strides=1, padding='same', activation='relu'),
            layers.MaxPooling2D(2, strides=2),
            layers.Conv2D(4, 3, strides=1, padding='same', activation='relu'),
            layers.MaxPooling2D(2, strides=2),
            layers.Conv2D(2, 3, strides=1, padding='same', activation='relu'),
            layers.Flatten(),
            layers.Dense(10,activation='relu'),
            layers.Dropout(0.3),
            layers.Dense(5,activation='relu')])

        self.combine = tf.keras.Sequential([
            layers.Input(shape=(20,)),
            layers.Dense(15, activation='relu'),
            layers.Dense(5, activation='relu'),
            layers.Dropout(0.3),
            layers.Dense(1, activation='sigmoid')])

    def call(self, x):
        front = self.front(x[0])
        left = self.left(x[1])
        right = self.right(x[2])

        out=self.combine(layers.Concatenate(axis=1)([front,left,right]))

        return out


## **Deep Learning**

In [30]:
epochs=30

In [31]:
with strategy.scope():
  cnn_combined=CNN_Views()
  cnn_combined.compile(optimizer='adam',  loss='binary_crossentropy',metrics=tf.keras.metrics.BinaryAccuracy(name='binary_accuracy', dtype=None, threshold=0.5))
history=cnn_combined.fit([xm_train_frontal, xm_train_left, xm_train_right],y_train_frontal, epochs=epochs,
      batch_size=16, shuffle=True)

Epoch 1/30
13/13 [==============================] - 12s 47ms/step - loss: 0.6870 - binary_accuracy: 0.5567
Epoch 2/30
13/13 [==============================] - 1s 47ms/step - loss: 0.6577 - binary_accuracy: 0.5979
Epoch 3/30
13/13 [==============================] - 1s 47ms/step - loss: 0.6560 - binary_accuracy: 0.5206
Epoch 4/30
13/13 [==============================] - 1s 47ms/step - loss: 0.6315 - binary_accuracy: 0.5567
Epoch 5/30
13/13 [==============================] - 1s 46ms/step - loss: 0.5909 - binary_accuracy: 0.7062
Epoch 6/30
13/13 [==============================] - 1s 46ms/step - loss: 0.5580 - binary_accuracy: 0.6649
Epoch 7/30
13/13 [==============================] - 1s 46ms/step - loss: 0.5511 - binary_accuracy: 0.7474
Epoch 8/30
13/13 [==============================] - 1s 48ms/step - loss: 0.5027 - binary_accuracy: 0.7526
Epoch 9/30
13/13 [==============================] - 1s 47ms/step - loss: 0.5724 - binary_accuracy: 0.7423
Epoch 10/30
13/13 [==========================

In [32]:
#,validation_data=([xm_test_frontal, xm_test_left, xm_test_right, xm_test_left45, xm_test_right45],y_test_frontal)

In [33]:
cnn_combined.evaluate([xm_test_frontal, xm_test_left, xm_test_right], y_test_frontal,return_dict=True)

3/3 [==============================] - 8s 1s/step - loss: 0.4107 - binary_accuracy: 0.8795


{'loss': 0.41066309809684753, 'binary_accuracy': 0.8795180916786194}

In [34]:
import seaborn as sns

In [35]:
y_pred=cnn_combined.predict([xm_test_frontal, xm_test_left, xm_test_right])

3/3 [==============================] - 7s 1s/step


In [36]:
y_pred=np.where(y_pred > 0.5, 1,0)

In [37]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay,classification_report

cm=confusion_matrix(y_test_frontal, y_pred)
print(cm)
print(classification_report(y_test_frontal, y_pred))

[[48  5]
 [ 5 25]]
              precision    recall  f1-score   support

         0.0       0.91      0.91      0.91        53
         1.0       0.83      0.83      0.83        30

    accuracy                           0.88        83
   macro avg       0.87      0.87      0.87        83
weighted avg       0.88      0.88      0.88        83



In [38]:
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['healthy','sick'])
sns.reset_orig()
disp.plot()